In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import save_image


In [ ]:

# -----------------------------
# 1. Hyperparameters
# -----------------------------
image_size = 128   # resize Cube+ images to 128x128
batch_size = 32
latent_dim = 128
epochs = 30
lr = 1e-3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:

# -----------------------------
# 2. Dataset Loader (Cube+)
# -----------------------------
# Put Cube+ dataset images inside: ./data/cube_plus/
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor()
])

dataset = datasets.ImageFolder(root="./data/cube_plus/", transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [ ]:

# -----------------------------
# 3. VAE Model
# -----------------------------
class VAE(nn.Module):
    def __init__(self, latent_dim):
        super(VAE, self).__init__()
        # Encoder
        self.enc_conv = nn.Sequential(
            nn.Conv2d(3, 32, 4, 2, 1),  # [B, 32, 64, 64]
            nn.ReLU(),
            nn.Conv2d(32, 64, 4, 2, 1), # [B, 64, 32, 32]
            nn.ReLU(),
            nn.Conv2d(64, 128, 4, 2, 1), # [B, 128, 16, 16]
            nn.ReLU(),
            nn.Conv2d(128, 256, 4, 2, 1), # [B, 256, 8, 8]
            nn.ReLU(),
        )
        self.fc_mu = nn.Linear(256 * 8 * 8, latent_dim)
        self.fc_logvar = nn.Linear(256 * 8 * 8, latent_dim)

        # Decoder
        self.fc_decode = nn.Linear(latent_dim, 256 * 8 * 8)
        self.dec_conv = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 4, 2, 1), # [B, 128, 16, 16]
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 4, 2, 1), # [B, 64, 32, 32]
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 4, 2, 1), # [B, 32, 64, 64]
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, 4, 2, 1),  # [B, 3, 128, 128]
            nn.Sigmoid()
        )

    def encode(self, x):
        h = self.enc_conv(x)
        h = h.view(h.size(0), -1)
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = self.fc_decode(z)
        h = h.view(-1, 256, 8, 8)
        return self.dec_conv(h)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

In [ ]:
# -----------------------------
# 4. Loss Function
# -----------------------------
def loss_function(recon_x, x, mu, logvar):
    recon_loss = F.mse_loss(recon_x, x, reduction='sum')  # reconstruction
    kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return recon_loss + kl_loss

In [ ]:
# -----------------------------
# 5. Training
# -----------------------------
vae = VAE(latent_dim).to(device)
optimizer = torch.optim.Adam(vae.parameters(), lr=lr)

for epoch in range(epochs):
    vae.train()
    train_loss = 0
    for imgs, _ in dataloader:
        imgs = imgs.to(device)
        optimizer.zero_grad()
        recon, mu, logvar = vae(imgs)
        loss = loss_function(recon, imgs, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {train_loss/len(dataloader.dataset):.4f}")
    # Save some reconstructions
    if (epoch+1) % 5 == 0:
        save_image(recon[:8].cpu(), f"recon_epoch{epoch+1}.png")
        save_image(imgs[:8].cpu(), f"original_epoch{epoch+1}.png")

In [ ]:
# -----------------------------
# 6. Save model
# -----------------------------
torch.save(vae.state_dict(), "vae_cubeplus.pth")
